<a href="https://colab.research.google.com/github/greyhound101/hardhat/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AIVenture0/yolov5.git

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [2]:
# check for the cloned repo
!ls -R

Streaming output truncated to the last 5000 lines.
hard_hat_workers2236.png.txt  hard_hat_workers4485.txt
hard_hat_workers2236.txt      hard_hat_workers4486.png.txt
hard_hat_workers2237.png.txt  hard_hat_workers4486.txt
hard_hat_workers2238.png.txt  hard_hat_workers4487.png.txt
hard_hat_workers2238.txt      hard_hat_workers4487.txt
hard_hat_workers2239.png.txt  hard_hat_workers4488.png.txt
hard_hat_workers2239.txt      hard_hat_workers4488.txt
hard_hat_workers2240.png.txt  hard_hat_workers4489.txt
hard_hat_workers2240.txt      hard_hat_workers448.png.txt
hard_hat_workers2241.png.txt  hard_hat_workers448.txt
hard_hat_workers2241.txt      hard_hat_workers4490.png.txt
hard_hat_workers2242.png.txt  hard_hat_workers4490.txt
hard_hat_workers2242.txt      hard_hat_workers4491.png.txt
hard_hat_workers2243.png.txt  hard_hat_workers4491.txt
hard_hat_workers2243.txt      hard_hat_workers4492.png.txt
hard_hat_workers2245.png.txt  hard_hat_workers4492.txt
hard_hat_workers2245.txt      hard_hat_work

In [3]:
# move all the files of YOLOv5 to current working directory
!mv yolov5/* ./


mv: cannot stat 'yolov5/*': No such file or directory


In [4]:
# check for all the files in the current working directory
!ls

annotations  hubconf.py  __pycache__	   train_batch0.jpg  weights
convertor    images	 README.md	   train_batch1.jpg  wheat0.yaml
data	     inference	 requirements.txt  train_batch2.jpg  yolov5
detect.py    labels.png  runs		   train.py	     yolov5x.yaml
Dockerfile   LICENSE	 sample_data	   tutorial.ipynb
gdrive	     models	 test.py	   utils


In [5]:
!pip install -r requirements.txt

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-ub36ta_t
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-ub36ta_t
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=265032 sha256=8d7d19cdfc1a03ac6c1349ce9b76fc98234fe97133052eb0b40b0ba7881142fb
  Stored in directory: /tmp/pip-ephem-wheel-cache-q0c6zwaz/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [7]:
import zipfile
with zipfile.ZipFile('/content/gdrive/MyDrive/HardHat_Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [8]:
import glob
a=glob.glob('/content/annotations/*')
a[0]

'/content/annotations/hard_hat_workers1692.xml'

In [9]:
import numpy as np
import pandas as pd

import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from pathlib import Path

import os
import os.path

import uuid
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from tqdm import tqdm
def get_train_df():
    ann_path, img_path='/content/annotations','/content/images'
    def get_file_list(root, file_type):
        return [os.path.join(directory_path, f) for directory_path, directory_name, 
                files in os.walk(root) for f in files if f.endswith(file_type)]
    
    ann_path_list = get_file_list(ann_path, '.xml')
    ann_list = []
    
    for a_path in tqdm(ann_path_list):
        root = ET.parse(a_path).getroot()
        ann = {}
        
        ann['uuid']  = uuid.uuid1().int
        ann['filename'] = root.find("./filename").text.split('.')[0]
        
        # image sizes
        ann['width'] = root.find("./size/width").text
        ann['height'] = root.find("./size/height").text
        ann['depth'] = root.find("./size/depth").text
        
        for obj in root.findall('object'):
            ann['xmin'] = int(obj.find("bndbox").find('xmin').text)
            ann['ymin'] = int(obj.find("bndbox").find('ymin').text)
            ann['xmax'] = int(obj.find("bndbox").find('xmax').text)
            ann['ymax'] = int(obj.find("bndbox").find('ymax').text)
            
            ann_list.append(ann)
        
    return pd.DataFrame(ann_list)
df=get_train_df()
df['classes']=0
df['x_center']=(df['xmin']+df['xmax'])/2
df['y_center']=(df['ymin']+df['ymax'])/2
df['w']=df['width']
df['h']=df['height']

100%|██████████| 4750/4750 [00:00<00:00, 7197.38it/s]


In [10]:
index = list(set(df.filename))
len(index)

4750

In [11]:
source = 'train'
import shutil as sh
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('filename')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('convertor/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('convertor/fold{}/labels/'.format(fold)+path2save)
            # print(name)
            # name=name._str.split('/')[-1]
            with open('/content/convertor/fold{}/labels/'.format(str(fold))+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/1024
                row = row.astype(str)
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('convertor/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('convertor/fold{}/images/{}'.format(fold,path2save))
            sh.copy("/content/images/{}".format(name)+'.png','/content/convertor/fold{}/images/{}/{}'.format(fold,path2save,name)+'.png')

100%|██████████| 4750/4750 [00:29<00:00, 161.22it/s]


In [ ]:
!python ./train.py --img 640 --batch 1 --epochs 10 --data /content/wheat0.yaml --cfg /content/yolov5x.yaml

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Namespace(adam=False, batch_size=1, bucket='', cache_images=False, cfg='/content/yolov5x.yaml', data='/content/wheat0.yaml', device='', epochs=10, evolve=False, img_size=[640], multi_scale=False, name='', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15109MB)

2021-07-14 20:27:18.823647: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/

             